In [9]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import *
from scipy.fftpack import fft2, ifft2
from scipy.integrate import solve_ivp
from matplotlib.animation import FuncAnimation
from numpy.polynomial.chebyshev import Chebyshev
from numpy.polynomial.chebyshev import chebval, chebfit
from scipy.linalg import solve_banded

In [2]:
Lx, Ly = 20, 20
n = 64
beta = 1
D1, D2 = 0.1, 0.1
m = 1
tspan = np.arange(0,4.5,0.5)

x2 = np.linspace(-Lx/2, Lx/2, n + 1)
x = x2[:n]
y2 = np.linspace(-Ly/2, Ly/2, n + 1)
y = y2[:n]
X, Y = np.meshgrid(x, y)
u = np.tanh(np.sqrt(X**2+Y**2))*np.cos(m*np.angle(X+1j*Y) - (np.sqrt(X**2+Y**2)))
v = np.tanh(np.sqrt(X**2+Y**2))*np.sin(m*np.angle(X+1j*Y) - (np.sqrt(X**2+Y**2)))


kx = (2*np.pi/Lx) * np.concatenate((np.arange(0, n/2), np.arange(-n/2, 0)))
ky = (2*np.pi/Ly) * np.concatenate((np.arange(0, n/2), np.arange(-n/2, 0)))
kx[0] = 1e-6
ky[0] = 1e-6
KX, KY = np.meshgrid(kx, ky)
K = KX**2 + KY**2


In [3]:
def spc_rhs(t, y, n, beta, K, D1, D2):
    U_hat = y[:n*n].reshape((n, n))
    V_hat = y[n*n:].reshape((n, n))

    U = np.real(ifft2(U_hat))
    V = np.real(ifft2(V_hat))

    A = U**2 + V**2
    lambda_A = 1 - A
    omega_A = -beta*A

    reaction_U = fft2(lambda_A * U - omega_A * V)
    reaction_V = fft2(omega_A * U + lambda_A * V)

    diffusion_U = -K * U_hat * D1
    diffusion_V = -K * V_hat * D2

    du_dt = reaction_U + diffusion_U
    dv_dt = reaction_V + diffusion_V

    rhs = np.hstack([du_dt.flatten(), dv_dt.flatten()])
    return rhs

U = fft2(u)
V = fft2(v)
wt = np.hstack([U.flatten(), V.flatten()])

wtsol = solve_ivp(
    spc_rhs, [tspan[0], tspan[-1]], wt, t_eval=tspan,
    args=(n, beta, K, D1, D2), method='RK45'
)

A1 = wtsol.y
A1.shape

(8192, 9)

In [4]:
A1

array([[ 24.94003847  +0.j        ,  12.73268299  +0.j        ,
         -1.38095598  +0.j        , ..., -64.02389647  +0.j        ,
        -67.76356741  +0.j        , -61.18058974  +0.j        ],
       [-18.55666362 -58.16631091j, -42.51586944 -46.91292244j,
        -60.80795253 -25.74803902j, ..., -26.39439597+113.08288984j,
          6.86544434+123.00045628j,  41.4436393 +110.05531209j],
       [-16.04755868 +32.82798293j, -22.03971648 -45.79777396j,
        -23.23089505-104.14171583j, ..., -25.03391682 -92.65273136j,
        -29.2936105  -40.95948731j, -31.3712619  +15.69868908j],
       ...,
       [ 24.73021466-566.77472255j,  34.94179045-331.37291658j,
         38.82924248 -49.7842318j , ...,   4.99619196+602.39629491j,
         -9.93322885+490.73690642j, -25.6299042 +281.79202092j],
       [ 25.33720124-361.63379183j,  43.00958768-453.71174572j,
         51.93221654-447.84156197j, ..., -30.76392977+266.44218727j,
        -58.45411318+429.16535845j, -74.0191717 +505.3153223j ]

In [176]:
# time_points = wtsol.t
# u_sol = A1[: (n)**2, :].reshape((n, n, -1))
# v_sol = A1[(n)**2 :, :].reshape((n, n, -1))

# for i, t in enumerate(time_points):
#     plt.figure()
#     plt.contourf(X, Y, u_sol[:, :, i], levels=20, cmap="viridis")
#     plt.colorbar()
#     plt.title(f"U at t={t}")
#     plt.xlabel("x")
#     plt.ylabel("y")
#     plt.show()

#     plt.figure()
#     plt.contourf(X, Y, v_sol[:, :, i], levels=20, cmap="plasma")
#     plt.colorbar()
#     plt.title(f"V at t={t}")
#     plt.xlabel("x")
#     plt.ylabel("y")
#     plt.show()

In [24]:
Lx, Ly = 20, 20
n = 30
domain = [-1, 1]
beta = 1
D1, D2 = 0.1, 0.1
m = 1
tspan = np.arange(0,4.5,0.5)

def cheb(N):
	if N==0: 
		D = 0.; x = 1.
	else:
		n = arange(0,N+1)
		x = cos(pi*n/N).reshape(N+1,1) 
		c = (hstack(( [2.], ones(N-1), [2.]))*(-1)**n).reshape(N+1,1)
		X = tile(x,(1,N+1))
		dX = X - X.T
		D = dot(c,1./c.T)/(dX+eye(N+1))
		D -= diag(sum(D.T,axis=0))
	return D, x.reshape(N+1)


D, x_cheb =  cheb(n)
D[n,:] = 0
D[0,:] = 0
Dxx = np.dot(D,D)/(20/2)**2
x_cheb *= 10
X, Y = np.meshgrid(x_cheb, x_cheb)
I = np.eye(len(Dxx))
L = kron(I,Dxx) + kron(Dxx,I)

N = (n+1)*(n+1)

u = np.tanh(np.sqrt(X**2+Y**2))*np.cos(m*np.angle(X+1j*Y) - (np.sqrt(X**2+Y**2)))
v = np.tanh(np.sqrt(X**2+Y**2))*np.sin(m*np.angle(X+1j*Y) - (np.sqrt(X**2+Y**2)))

In [25]:
def spc_rhs2(t, y, n, beta, D, D1, D2):
    U = y[0:N]
    V = y[N:]

    A = U**2 + V**2
    lambda_A = 1 - A
    omega_A = -beta * A

    reaction_U = lambda_A * U - omega_A * V
    reaction_V = omega_A * U + lambda_A * V

    diffusion_U = D1 * np.dot(L,U)
    diffusion_V = D2 * np.dot(L,V)

    du_dt = reaction_U + diffusion_U
    dv_dt = reaction_V + diffusion_V

    rhs = np.hstack([du_dt.flatten(), dv_dt.flatten()])
    return rhs

wt2 = np.hstack([u.flatten(), v.flatten()])
wtsol2 = solve_ivp(
    spc_rhs2, [tspan[0], tspan[-1]], wt2, t_eval=tspan,
    args=(n, beta, D, D1, D2), method='RK45'
)

A2 = wtsol2.y
A2
# def plot_no_flux(sol, x, title):
#     n = len(x)
#     U = sol.y[:n * n, -1].reshape((n, n))
#     plt.contourf(x, x, U, levels=50, cmap='viridis')
#     plt.colorbar()
#     plt.title(title)
#     plt.xlabel('x')
#     plt.ylabel('y')
#     plt.show()

# plot_no_flux(wtsol2, x_cheb, "No-Flux Boundaries")

array([[ 0.70358468,  0.27678435, -0.21775865, ..., -0.79689015,
        -0.40972859,  0.07776933],
       [ 0.73241275,  0.47188952,  0.07344742, ..., -0.96577657,
        -0.78500366, -0.4261521 ],
       [ 0.81058026,  0.37605887, -0.11123233, ..., -0.84008598,
        -0.49565779, -0.03085913],
       ...,
       [ 0.58562756,  0.91352592,  0.97914313, ..., -0.50294695,
        -0.84298442, -0.97634716],
       [ 0.6808609 ,  0.87018536,  0.97997159, ..., -0.16453512,
        -0.5878894 , -0.88455009],
       [ 0.71061143,  0.96093661,  0.97601586, ..., -0.60413504,
        -0.91222169, -0.99697897]])

In [23]:
A2.shape

(1922, 9)